In [1]:
from collections import namedtuple

In [2]:
Result = namedtuple('Result', 'count average')

#### # the subgenerator

In [3]:
def averager():  # <1>
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield  # <2>
        if term is None:  # <3>
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)  # <4

#### # the delegating generator

In [4]:
def grouper(results, key):  # <5>
    while True:  # <6>
        results[key] = yield from averager()  # <7>

#### # the client code, a.k.a. the caller

In [5]:
def main(data):  # <8>
    results = {}
    for key, values in data.items():
        group = grouper(results, key)  # <9>
        next(group)  # <10>
        for value in values:
            group.send(value)  # <11>
        group.send(None)  # important! <12>

    # print(results)  # uncomment to debug
    report(results)

#### # output report

In [6]:
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))

In [7]:
data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

In [8]:
if __name__ == '__main__':
    main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


#### A coroutine to compute a running average.
#### Testing ``averager`` by itself::

In [9]:
>>> coro_avg = averager()
>>> next(coro_avg)
>>> coro_avg.send(10)
>>> coro_avg.send(30)
>>> coro_avg.send(6.5)
>>> coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

#### Driving it with ``yield from``::

In [10]:
def summarize(results):
     while True:
            result = yield from averager()
            results.append(result)

In [11]:
results = []

In [12]:
summary = summarize(results)
next(summary)
for height in data['girls;m']:
     summary.send(height)

In [13]:
summary.send(None)
for height in data['boys;m']:
     summary.send(height)

In [14]:
>>> summary.send(None)
>>> results == [
...     Result(count=10, average=1.4279999999999997),
...     Result(count=9, average=1.3888888888888888)
... ]

True